In [1]:
using LinearAlgebra
using DynamicPolynomials
using TSSOS
using JuMP
using Random
using QuadGK
using NLopt


According to Section *State-transfer in linear networks* of [Kurt Jacobs *et al* 2016 EPL **114** 40007](https://iopscience.iop.org/article/10.1209/0295-5075/114/40007#epl17883s5), the equation of the motion for joint second moments
$$
    C = \langle {\bf v}^t {\bf v} \rangle 
    = \begin{pmatrix}
        \langle a^2 \rangle & \langle  a a^{\dagger} \rangle & \langle ab \rangle & \langle ab^{\dagger} \rangle \\
        \langle  a^\dagger a \rangle & \langle a^{\dagger} a^{\dagger} \rangle & \langle a^{\dagger} b \rangle & \langle a^{\dagger} b^{\dagger} \rangle \\
        \langle ab \rangle & \langle a^{\dagger} b \rangle & \langle b^2 \rangle & \langle b b^{\dagger} \rangle \\
        \langle a b^{\dagger} \rangle & \langle a^{\dagger} b^{\dagger} \rangle & \langle b^{\dagger} b \rangle & \langle b^{\dagger} b^{\dagger} \rangle 
     \end{pmatrix}
$$
is given by
$$
    \partial_t C = AC + CA^T.
$$

$$
    \partial_t vec(C) = (A ⊗ 1 + 1 ⊗ A) vec(C) 
$$

In [2]:
# the initial condition
vecC₀ = [0; 1; 0; 0;   2; 0; 0; 0;   0; 0; 0; 0;   0; 0; 1; 0];

In [3]:
# systems parameters
ω = 1.
g = 1.



function A(λ₁)
    # The matrix in the equation of motion for C
    
    im * [ω    0      -λ₁*g  -λ₁*g
          0    -ω     λ₁*g  λ₁*g
         -λ₁*g -λ₁*g  ω      0
          λ₁*g λ₁*g   0     -ω]
end

A (generic function with 1 method)

## Utilities for working with Polynomials

In [4]:
function ∫(p::AbstractPolynomial, x::PolyVar, x_lower, x_upper)
    
    # get the index of the variable of integration
    ind_x = indexin([x], variables(p))[1]
        
    if isnothing(ind_x)
        # integration valuable is not found among vars
        return p * (x_upper - x_lower)
    end
    
    # get the indefinite integral
    int_p = sum(
        term * x * 1 // (exponents(term)[ind_x] + 1) for term in terms(p)
        init = 0 * x
    )
            
    # get the definite integral
    subs(int_p, x=>x_upper) - subs(int_p, x=>x_lower)
end

function ∫(M::AbstractMatrix, x::PolyVar, x_lower, x_upper)
   map(z -> ∫(z, x, x_lower, x_upper), M) 
end

function real_poly(p::Polynomial)
    #=
    Real part of the polynomial
    =#
    sum(
        real(c) * m for (c, m) in zip(coefficients(p), monomials(p))# if ~isapproxzero(abs(c))
    )
end

function square_frobenius_norm(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    real_poly(sum(z' * z for z in M))
end

square_frobenius_norm (generic function with 1 method)

## Chebyshve polynomial approximation for $\exp_p(\Delta t \Omega)$

In [5]:
using SpecialFunctions

"""
Chebyshev approximation for exp(Δt Ω)
"""
function exp_chebyshev(Δt::Real, Ω::AbstractMatrix, order::Integer)
    
    Tₙ₋₁ = I
    Tₙ  = Ω
    
    # The first two terms of Chebyshev series for exp
    series = besselj(0, Δt) * Tₙ₋₁ + 2 * besselj(1, Δt) * Tₙ
    
    for n=2:order
        Tₙ₊₁  = 2 * Ω * Tₙ + Tₙ₋₁
        
        series .+= 2 * besselj(n, Δt) * Tₙ₊₁
        
        (Tₙ, Tₙ₋₁) = (Tₙ₊₁, Tₙ) 
    end
    
    series
end

exp_chebyshev

In [6]:
@polyvar λ[1:10]

T = 0.2

Δt = T / size(λ)[1]

function 𝓐(x)
    #=
    The generator of motion
    =#
    a = A(x)
    
    𝓘 = Matrix(I, size(a))
    
    kron(a, 𝓘) + kron(𝓘, a)
end

obj_propagator = prod(exp_chebyshev(Δt, 𝓐(λ₁), 2) for λ₁ in λ);

In [7]:
function local_minimize(obj::AbstractPolynomial, init_x::AbstractArray)
    #=
    Perform local minimization of obj polynomial using init_x as initial guess
    =#
    vars = variables(obj)

    @assert length(vars) == length(init_x)
    
    function g(a...)
        # Converting polynomial expression to function to be minimize
        obj(vars => a)
    end
    
    model = Model(NLopt.Optimizer)

    set_optimizer_attribute(model, "algorithm", :LD_MMA)

    set_silent(model)
    @variable(model, y[1:length(vars)])

    # set initial guess
    for (var, init_val) in zip(y, init_x)
        set_start_value(var, init_val)
    end

    register(model, :g, length(y), g; autodiff = true)
    @NLobjective(model, Min, g(y...))
    JuMP.optimize!(model)

    map(value, y)
end


function propagate(x::AbstractArray)   
    #=
    Solve the equation of motion
    =#
    prod(exp(Δt * 𝓐(λ₁)) for λ₁ in x) * vecC₀
end

function symbolic_propagation(x::AbstractArray)   
    map(z -> z(x), obj_propagator) * vecC₀
end

symbolic_propagation (generic function with 1 method)

### Construct the objective function $\langle  a^\dagger a \rangle$ 

In [8]:
obj = (obj_propagator * vecC₀)[2]
obj = real_poly(obj);




function clip(z)
    sum(T for T in terms(z) if maxdegree(T) ≤ 3)
end

obj = map(clip, obj_propagator)

obj = (obj * vecC₀)[2]

obj = real_poly(obj);

In [9]:
# Get the global minimum via TSSOS library
#opt,sol,data = tssos_first(obj, variables(obj); QUIET = true, solution = true)
opt,sol,data = tssos_first(
    [obj; [ξ + 1 for ξ in λ]; [-ξ + 1 for ξ in λ]], 
    variables(obj), maxdegree(obj) ÷ 2, numeq=0; QUIET = true, solution = true
)

previous_sol = sol
previous_opt = opt

while ~isnothing(sol)
    previous_sol = sol
    previous_opt = opt
            
    opt,sol,data = tssos_higher!(data; QUIET = true, solution = true)
end


tssos_glob_obj_min = previous_opt
glob_min_x = previous_sol

***************************TSSOS***************************
TSSOS is launching...


LoadError: SYSTEM: show(lasterr) caused an error

In [10]:
real(propagate(glob_min_x)[2]), real(symbolic_propagation(glob_min_x)[2])

LoadError: UndefVarError: glob_min_x not defined

In [11]:
maximum(abs.(propagate(glob_min_x) - symbolic_propagation(glob_min_x)))

LoadError: UndefVarError: glob_min_x not defined

In [12]:
propagate(glob_min_x)

LoadError: UndefVarError: glob_min_x not defined

In [13]:
2*π / 20

0.3141592653589793

In [14]:
Δt * maximum(abs.(eigvals(𝓐(3))))

0.10583005244258371

In [15]:
besselj(1, Δt)

0.009999500008333264